In [1]:
!pip install hmmlearn
import numpy as np
from hmmlearn import hmm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 2.9 MB/s eta 0:00:00


In [2]:
states = ["Dificil", "Mediu", "Usor"]
observations = ["FB", "B", "S", "NS"]

start_prob = np.array([1/3, 1/3, 1/3])

trans_mat = np.array([
    [0.0, 0.5, 0.5],
    [0.5, 0.25, 0.25],
    [0.5, 0.25, 0.25]
])

emission_prob = np.array([
    [0.1, 0.2, 0.4, 0.3],
    [0.15, 0.25, 0.5, 0.1],
    [0.2, 0.3, 0.4, 0.1]
])

model = hmm.CategoricalHMM(n_components=3)
model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

In [3]:
obs_seq = np.array([0, 0, 2, 1, 1, 2, 1, 1, 3, 1, 1]).reshape(-1, 1)

log_prob = model.score(obs_seq)
print(np.exp(log_prob))

1.6888145197875796e-07


In [4]:
log_prob_path, path = model.decode(obs_seq)
print([states[i] for i in path])
print(np.exp(log_prob_path))

['Usor', 'Usor', 'Dificil', 'Usor', 'Dificil', 'Mediu', 'Dificil', 'Usor', 'Dificil', 'Usor', 'Dificil']
8.437500000000068e-11


In [5]:
def viterbi_algorithm(obs, pi, A, B):
    N = A.shape[0]
    T = len(obs)

    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    delta[0] = np.log(pi + 1e-10) + np.log(B[:, obs[0]] + 1e-10)

    for t in range(1, T):
        for j in range(N):
            prob_transitions = delta[t-1] + np.log(A[:, j] + 1e-10)
            psi[t, j] = np.argmax(prob_transitions)
            delta[t, j] = np.max(prob_transitions) + np.log(B[j, obs[t]] + 1e-10)

    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(delta[-1])

    for t in range(T-2, -1, -1):
        path[t] = psi[t+1, path[t+1]]

    return path

manual_path = viterbi_algorithm(obs_seq.flatten(), start_prob, trans_mat, emission_prob)
print([states[i] for i in manual_path])

['Dificil', 'Usor', 'Dificil', 'Usor', 'Dificil', 'Mediu', 'Dificil', 'Usor', 'Dificil', 'Usor', 'Dificil']
